Practica 8  Aplicacion de redes neuronales


In [1]:
#Imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [2]:
#LOAD_DB
columns_names = ['Target','Alcohol','Malic acid','Ash','Alcalinity of ash',
                 'Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols',
                 'Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines',
                 'Proline']
ds = pd.read_csv('../datasets/Wine/wine.csv', header=-1, names=columns_names)

In [3]:
#PLOT_CONFUSION_MATRIX [Funcion extra]
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.tight_layout()

In [4]:
X = ds.drop(columns=['Target'])
#X.describe()


In [5]:
y = ds['Target']
#y.describe()

In [6]:
#SCALER 
scaler = StandardScaler()
scaler.fit(ds)
df_corr = pd.DataFrame(scaler.transform(ds), columns = ds.columns)
corr = df_corr.corr()
plt.figure(figsize=(25,11))
#sns.heatmap(corr, cmap = 'inferno')
#plt.show()


<Figure size 1800x792 with 0 Axes>

<Figure size 1800x792 with 0 Axes>

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import itertools


In [11]:
#MODEL_DEFINITION 
def get_model():
    model = Sequential()
    #Capa de entrada y oculta
    model.add(Dense(1, input_shape=(nb_inputs,), kernel_initializer='random_uniform'))
    N = 10
    #CAPAS
    model.add(Dense(N, activation ='exponential'))
    model.add(Dense(N, ))
    #model.add(Dense(N, ))
    #model.add(Dense(N, ))
    
    
    #Capa de salida
    #Neuronas de salida igual a la cantidad de etiquetas
    model.add(Dense(nb_targets))
    model.add(Activation('softmax'))
    
    #Método de aprendizaje
    #opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)#Stochastic gradient descent 
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#ADAM
    #opt = Adagrad(lr=0.01, epsilon=None, decay=0.0)
    #opt = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
    #opt = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    #opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    

    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    return model

In [12]:
plt.figure()


#kf = StratifiedKFold(n_splits=5)
nb_inputs = len(list(X.columns))
nb_targets = len(list(y.unique()))

X[:] = scaler.fit_transform(X.loc[:])
#NUM_ITER

kf = StratifiedKFold(n_splits=4)
i = 0
avg_acc = []
avg_f1 = []
for idx_train, idx_test in kf.split(X,y):
    X_train = X.loc[idx_train]
    X_test = X.loc[idx_test]
    y_train = y.loc[idx_train]
    y_test = y.loc[idx_test]

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    nn_model = get_model()


    #pd.get_dummies(y_train) Crea representación One-hot-encoding
    
    Niter = 25
    for PN in (range(0,Niter)):
        nn_model.fit(X_train, pd.get_dummies(y_train), epochs=150, verbose=False)
        #print("{} ".format((PN+1)/Niter))

    y_pred = nn_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)+1
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    avg_acc.append(acc)
    avg_f1.append(f1)
    print("Fold {}\nAccuracy={:.3f}\nF1 score={:.3}\n".format(i+1,acc,f1))
    #plot_confusion_matrix(confusion_matrix(y_test, y_pred), classes=['Low', 'Medium', 'High'],
    #                  title='Confusion matrix, without normalization')
    #plt.show()
    
    
    i = i+1

print("AVERAGE\nAccuracy={:.3f}\nF1 score={:.3}\n".format(sum(avg_acc)/len(avg_acc),
                                                          sum(avg_f1)/len(avg_f1)))



Fold 1
Accuracy=0.889
F1 score=0.889

Fold 2
Accuracy=0.911
F1 score=0.911

Fold 3
Accuracy=0.911
F1 score=0.911

Fold 4
Accuracy=0.953
F1 score=0.953

AVERAGE
Accuracy=0.916
F1 score=0.916



<Figure size 432x288 with 0 Axes>

<img src='Classificacion_Bitacora.JPG'>